In [222]:
##import and set up

import pytorch_lightning as pl
import torch
from torchtext import data
import spacy
from torchtext import datasets
import os
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from models.simple_rnn import RNN, CNN
import lime
from xai.shap import xai_shap

from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization, Lime, LayerGradientShap, LayerDeepLiftShap, LayerDeepLift
from xai.visualize import interpret_sentence

nlp = spacy.load('en')


torch.backends.cudnn.deterministic = True
device='cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [158]:
#seed for reproducibility

SEED = 0

torch.manual_seed(SEED)


In [233]:
#for visualizing and interpreting

def forward_with_sigmoid(model,input):
    # print(torch.sigmoid(model(input)))
    return torch.sigmoid(model(input.to('cuda')))

# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence,  TEXT, Label, min_len = 7, label = 0):
    PAD_IND = TEXT.vocab.stoi['pad']
    token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
    lig = Lime(model)
    # lig = LayerIntegratedGradients(model, model.embedding)
    # lig = LayerGradientShap(model, model.embedding)
    # lig = LayerDeepLift(model, model.embedding)


    text = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(text) < min_len:
        text += ['pad'] * (min_len - len(text))
    indexed = [TEXT.vocab.stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device, dtype=torch.long)
    input_indices = input_indices.unsqueeze(0)

    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(model,input_indices)[0,0].item()
    # print("Pred : ", pred)
    pred_ind = round(pred)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices)

    print('pred: ', Label.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig, TEXT, Label)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records, TEXT, Label ):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            Label.vocab.itos[pred_ind],
                            Label.vocab.itos[label],
                            Label.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))



In [197]:
#data set up
from torchtext import vocab
import torchtext
 

TEXT = torchtext.data.Field(lower=True, tokenize='spacy', batch_first=True)
Label = torchtext.data.LabelField(dtype = torch.long)

train_data, test_data = datasets.IMDB.splits(TEXT, Label)

train_data, valid_data = train_data.split()
MAX_VOCAB_SIZE = 25000
# If you prefer to use pre-downloaded glove vectors, you can load them with the following two command line
loaded_vectors = torchtext.vocab.Vectors('C:/Users/Abhishek Saroha/Documents/GitHub/beyond-simple-word-level-input-relevance-explanations/data/glove.6B.50d.txt')
TEXT.build_vocab(train_data, vectors=loaded_vectors, max_size=len(loaded_vectors.stoi))
# TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, vectors = "glove.6B.100d", unk_init = torch.Tensor.normal_)
 
TEXT.vocab.set_vectors(stoi=loaded_vectors.stoi, vectors=loaded_vectors.vectors, dim=loaded_vectors.dim)
Label.build_vocab(train_data)

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
        (train_data, valid_data, test_data), 
        batch_size = 64,
        device = device)

PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)



In [234]:
INPUT_DIM = len(TEXT.vocab)
INPUT_DIM = 84970
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model_self = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

# trainer = pl.Trainer(max_epochs=5, gpus=1)
# trainer.fit(model_self, train_iterator)
model_self.load_state_dict(torch.load("./models/model_cnn_self_imdb.pt"))
model = model_self

# model.load_state_dict(torch.load('models/imdb-model-cnn.pt'))
model.eval()
model = model.to('cuda') 

In [235]:
#examples to interpret
interpret_sentence(model, 'It was a fantastic performance !', TEXT=TEXT, Label=Label, label=1)
interpret_sentence(model, 'Best film ever', TEXT=TEXT, Label=Label, label=1)
interpret_sentence(model, 'Such a great show!', TEXT=TEXT, Label=Label, label=1)
interpret_sentence(model, 'It was a horrible movie', TEXT=TEXT, Label=Label, label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', TEXT=TEXT, Label=Label, label=0)
interpret_sentence(model, 'It is a disgusting movie!', TEXT=TEXT, Label=Label, label=0)

RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)

In [226]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)
vis_data_records_ig = [] #to clear for future references

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,neg (0.38),pos,-1.94,It was a fantastic performance ! pad
pos,pos (0.88),pos,-0.67,Best film ever pad pad pad pad
pos,neg (0.16),pos,-1.88,Such a great show ! pad pad
neg,pos (0.83),pos,-0.56,It was a horrible movie pad pad
neg,neg (0.03),pos,-1.85,I 've never watched something as bad
neg,neg (0.33),pos,-2.24,It is a disgusting movie ! pad


In [227]:
model = torch.load('./models/imdb-model-cnn.pt')
model.eval()
model = model.to('cuda') 

In [228]:
#examples to interpret
interpret_sentence(model, 'It was a fantastic performance !', TEXT=TEXT, Label=Label, label=1)
interpret_sentence(model, 'Best film ever', TEXT=TEXT, Label=Label, label=1)
interpret_sentence(model, 'Such a great show!', TEXT=TEXT, Label=Label, label=1)
interpret_sentence(model, 'It was a horrible movie', TEXT=TEXT, Label=Label, label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', TEXT=TEXT, Label=Label, label=0)
interpret_sentence(model, 'It is a disgusting movie!', TEXT=TEXT, Label=Label, label=0)

pred:  pos ( 0.56 ) , delta:  tensor([0.5090], device='cuda:0')
pred:  pos ( 0.50 ) , delta:  tensor([0.2917], device='cuda:0')
pred:  pos ( 0.56 ) , delta:  tensor([0.9208], device='cuda:0')
pred:  neg ( 0.50 ) , delta:  tensor([0.0890], device='cuda:0')
pred:  pos ( 0.97 ) , delta:  tensor([11.7807], device='cuda:0')
pred:  neg ( 0.47 ) , delta:  tensor([0.3219], device='cuda:0')


In [229]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)
vis_data_records_ig = [] #to clear for future references

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.56),pos,1.28,It was a fantastic performance ! pad
pos,pos (0.50),pos,-0.21,Best film ever pad pad pad pad
pos,pos (0.56),pos,1.75,Such a great show ! pad pad
neg,neg (0.50),pos,0.01,It was a horrible movie pad pad
neg,pos (0.97),pos,1.89,I 've never watched something as bad
neg,neg (0.47),pos,-0.21,It is a disgusting movie ! pad


In [63]:

##running with our basic 50% rnn
#call model
model_rnn = RNN()

#get data
text, labels, train_data, test_data, valid_data, train_iterator, valid_iterator, test_iterator = model_rnn.preprocess_data(device, BATCH_SIZE=64)

# init model
INPUT_DIM = len(text.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model_rnn.create_model(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

model_rnn.load_state_dict(torch.load("./models/model_imdb.pt"))

model_rnn.eval()



RNN(
  (embedding): Embedding(20002, 100)
  (rnn): LSTM(100, 256)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (output): Softmax(dim=None)
)

In [78]:
interpret_sentence(model_rnn, 'It was a fantastic performance !', TEXT=text, Label=labels, label=1)
interpret_sentence(model_rnn, 'Best film ever', TEXT=text, Label=labels, label=1)
interpret_sentence(model_rnn, 'Such a great show!', TEXT=text, Label=labels, label=1)
interpret_sentence(model_rnn, 'It was a horrible movie', TEXT=text, Label=labels, label=0)
interpret_sentence(model_rnn, 'I\'ve never watched something as bad', TEXT=text, Label=labels, label=0)
interpret_sentence(model_rnn, 'It is a disgusting movie!', TEXT=text, Label=labels, label=0)

pred:  pos ( 0.73 ) , delta:  0
pred:  pos ( 0.73 ) , delta:  0
pred:  pos ( 0.73 ) , delta:  0
pred:  pos ( 0.73 ) , delta:  0
pred:  pos ( 0.73 ) , delta:  0
pred:  pos ( 0.73 ) , delta:  0


In [73]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)
vis_data_records_ig = [] #to clear for future references

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.73),pos,nan,It was a fantastic performance ! pad
pos,pos (0.73),pos,nan,Best film ever pad pad pad pad
pos,pos (0.73),pos,nan,Such a great show ! pad pad
neg,pos (0.73),pos,nan,It was a horrible movie pad pad
neg,pos (0.73),pos,nan,I 've never watched something as bad
neg,pos (0.73),pos,nan,It is a disgusting movie ! pad
